## Autoencoders

### Simple AE
MNIST is used as dataset, the input is binarized and Binary Cross Entropy is used as the loss function

In [1]:
import os

import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms 
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [2]:
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

def to_img(x):
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [3]:
num_epochs = 10
batch_size = 128
learning_rate = 1e-3

In [4]:
def plot_sample_img(img, name):
    img = img.view(1, 28, 28)
    save_image(img, './sample_{}.png'.format(name))

In [5]:
def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor

In [6]:
def tensor_round(tensor):
    return torch.round(tensor)

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:tensor_round(tensor))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 256),
            nn.ReLU(True),
            nn.Linear(256, 64),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(True),
            nn.Linear(256, 28 * 28),
            nn.Sigmoid())
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
#with torch.cuda.device(2):
cuda2 = torch.device('cuda:2')
model = autoencoder().cuda(cuda2)
criterion = nn.BCELoss() ##Binary Cross Entropy
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda(cuda2)
        #==========forward=============
        output = model(img)
        loss = criterion(output, img)
        MSE_loss = nn.MSELoss()(output, img)
        #==========backward============
        optimizer.zero_grad()
        loss.backward()
        optimizer.step
        #=================log======================
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data[0], MSE_loss.data[0]))
    if epoch % 10 == 0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        save_image(x, './mlp_img/x_{}.png'.format(epoch))
        save_image(x_hat, './mlp_img/x_hat_{}.png'.format(epoch))

In [8]:
for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        print(img.shape)

torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size([128, 784])
torch.Size(

In [9]:
28*28

784

### Denoising AE
We add random noise to the data and create a corrupted input (e.g. 20% noise has been added).

In [1]:
import os

import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [2]:
if not os.path.exists('./DAEmlp_img'):
    os.mkdir('./DAEmlp_img')

In [3]:
def to_img(x):
    x = x.view(x.size(0), 1, 28, 28)
    return x

num_epochs = 50
batch_size = 128
learning_rate = 1e-3

In [4]:
def add_noise(img):
    noise = torch.randn(img.size()) * 0.4
    noisy_img = img + noise
    return noisy_img

In [5]:
def plot_sample_img(img, name):
    img = img.view(1, 28, 28)
    save_image(img, './sample_{}.png'.format(name))

In [6]:
def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


def tensor_round(tensor):
    return torch.round(tensor)


img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:tensor_round(tensor))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 256),
            nn.ReLU(True),
            nn.Linear(256, 64),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(True),
            nn.Linear(256, 28 * 28),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

torch.cuda.set_device(2)
model = autoencoder().cuda()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        noisy_img = add_noise(img)
        noisy_img = Variable(noisy_img).cuda()
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(noisy_img)
        loss = criterion(output, img)
        MSE_loss = nn.MSELoss()(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data[0], MSE_loss.data[0]))
    if epoch % 10 == 0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        x_noisy = to_img(noisy_img.cpu().data)
        weights = to_img(model.encoder[0].weight.cpu().data)
        save_image(x, './DAEmlp_img/x_{}.png'.format(epoch))
        save_image(x_hat, './DAEmlp_img/x_hat_{}.png'.format(epoch))
        save_image(x_noisy, './DAEmlp_img/x_noisy_{}.png'.format(epoch))
        save_image(weights, './filters/epoch_{}.png'.format(epoch))


/data1/users/isotta/multMyeloma/bin_pytorch/dl4ehr/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/50], loss:0.1262, MSE_loss:0.0390
epoch [2/50], loss:0.0994, MSE_loss:0.0304
epoch [3/50], loss:0.0907, MSE_loss:0.0278
epoch [4/50], loss:0.0799, MSE_loss:0.0244
epoch [5/50], loss:0.0816, MSE_loss:0.0250
epoch [6/50], loss:0.0804, MSE_loss:0.0244
epoch [7/50], loss:0.0753, MSE_loss:0.0227
epoch [8/50], loss:0.0719, MSE_loss:0.0219
epoch [9/50], loss:0.0728, MSE_loss:0.0222
epoch [10/50], loss:0.0743, MSE_loss:0.0225
epoch [11/50], loss:0.0737, MSE_loss:0.0223
epoch [12/50], loss:0.0732, MSE_loss:0.0221
epoch [13/50], loss:0.0663, MSE_loss:0.0201
epoch [14/50], loss:0.0730, MSE_loss:0.0221
epoch [15/50], loss:0.0716, MSE_loss:0.0215
epoch [16/50], loss:0.0692, MSE_loss:0.0207
epoch [17/50], loss:0.0666, MSE_loss:0.0202
epoch [18/50], loss:0.0714, MSE_loss:0.0216
epoch [19/50], loss:0.0685, MSE_loss:0.0205
epoch [20/50], loss:0.0701, MSE_loss:0.0211
epoch [21/50], loss:0.0666, MSE_loss:0.0200
epoch [22/50], loss:0.0644, MSE_loss:0.0194
epoch [23/50], loss:0.0691, MSE_loss:0.02

In [8]:
torch.cuda.empty()

AttributeError: module 'torch.cuda' has no attribute 'empty'

In [9]:
next(model.parameters()).is_cuda

True